In [2]:
import os
import numpy as np
from utils import model_utils

from signlens.params import *
from signlens.preprocessing.data import load_data_subset_csv, load_glossary
from signlens.preprocessing.preprocess import group_pad_sequences, decode_labels


# from signlens.preprocessing.registry import load_model # to be deleted
%load_ext autoreload

%autoreload 2


In [3]:
test_data = load_data_subset_csv(balanced=True, csv_path=TRAIN_TEST_CSV_PATH)


Loading data subset from train_test.csv
✅ Filtered on n_frames = 100. Size reduced from 17233 to 17233 (100.0%)
✅ Filtered on n_classes = 10. Size reduced from 17233 to 690 (4.0%)
✅ Balanced data, with average of 69.0 elements per class. Size reduced from 690 to 690 (100.0%)
✅ Loaded 690 rows (4.0% of the original 17233 rows) from the dataset.


In [4]:
test_data[test_data.sign == 'book']

,path,participant_id,sequence_id,sign,n_frames,n_frames2,file_path
2,train_landmark_files_noface/22343/1963578447.p...,22343,1963578447,book,27.0,27.0,/home/bfrisque/code/benoitfrisque/signlens/raw...
10,train_landmark_files_noface/32319/2058774192.p...,32319,2058774192,book,6.0,6.0,/home/bfrisque/code/benoitfrisque/signlens/raw...
19,train_landmark_files_noface/62590/2405315892.p...,62590,2405315892,book,7.0,7.0,/home/bfrisque/code/benoitfrisque/signlens/raw...
29,train_landmark_files_noface/27610/40068588.par...,27610,40068588,book,6.0,6.0,/home/bfrisque/code/benoitfrisque/signlens/raw...
42,train_landmark_files_noface/34503/2067083603.p...,34503,2067083603,book,8.0,8.0,/home/bfrisque/code/benoitfrisque/signlens/raw...
...,...,...,...,...,...,...,...
630,train_landmark_files_noface/62590/279698882.pa...,62590,279698882,book,15.0,15.0,/home/bfrisque/code/benoitfrisque/signlens/raw...
657,train_landmark_files_noface/16069/405329692.pa...,16069,405329692,book,21.0,21.0,/home/bfrisque/code/benoitfrisque/signlens/raw...
669,train_landmark_files_noface/26734/2197755065.p...,26734,2197755065,book,6.0,6.0,/home/bfrisque/code/benoitfrisque/signlens/raw...
673,train_landmark_files_noface/26734/2757335993.p...,26734,2757335993,book,28.0,28.0,/home/bfrisque/code/benoitfrisque/signlens/raw...


In [5]:
test_subset = test_data.iloc[0:2]
test_subset

,path,participant_id,sequence_id,sign,n_frames,n_frames2,file_path
0,train_landmark_files_noface/36257/2228955138.p...,36257,2228955138,chair,42.0,42.0,/home/bfrisque/code/benoitfrisque/signlens/raw...
1,train_landmark_files_noface/2044/1158372807.pa...,2044,1158372807,go,6.0,6.0,/home/bfrisque/code/benoitfrisque/signlens/raw...


In [6]:
model_path = '../../training_outputs/model 20240322-145823/model/model_20240322-150323.keras'
model_name_folder = 'model 20240322-145823'


X_pred = group_pad_sequences(test_subset.file_path)


model = model_utils.load_model(model_name_folder=model_name_folder)
model = model[0]

y_pred =  model.predict(X_pred)
y_pred


/home/bfrisque/.pyenv/versions/3.10.6/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
2024-03-22 18:09:20.558278: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-22 18:09:20.558960: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...



Load latest model from local registry...

Load latest model from disk...
✅ Model loaded from local disk /home/bfrisque/code/benoitfrisque/signlens/training_outputs/model 20240322-145823
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step


array([[0.08138052, 0.16202429, 0.11876995, 0.13367637, 0.03784593,
        0.07303199, 0.15496674, 0.10007465, 0.04038617, 0.09784337],
       [0.1038068 , 0.14089708, 0.11039874, 0.12503351, 0.03557088,
        0.07659918, 0.14540334, 0.10983478, 0.0354893 , 0.11696641]],
      dtype=float32)

In [7]:
y_pred 

array([[0.08138052, 0.16202429, 0.11876995, 0.13367637, 0.03784593,
        0.07303199, 0.15496674, 0.10007465, 0.04038617, 0.09784337],
       [0.1038068 , 0.14089708, 0.11039874, 0.12503351, 0.03557088,
        0.07659918, 0.14540334, 0.10983478, 0.0354893 , 0.11696641]],
      dtype=float32)

In [8]:
decode_labels(y_pred)

(['drink', 'fine'], array([0.16202429, 0.14540334], dtype=float32))